# Game Summary

In [1]:
import requests
from datetime import timedelta, date, datetime
import json
import pandas as pd
import os
import csv

In [2]:
def daterange(start_date):
    end_date = datetime.now() - timedelta(days=1)
    current_date = datetime(start_date.year, start_date.month, start_date.day)
    one_day = timedelta(days=1)
    while current_date <= end_date:
        if current_date.month not in [1, 2, 11, 12]:
            yield current_date
        current_date += one_day

In [3]:
def get_daily_game_ids(api_key, access_level, language_code, year, month, day):
    ''' Fetches game IDs from the daily summary endpoint. '''
    try:
        # Ensure month and day are integers and format them to two digits
        month_str = f"{int(month):02d}"
        day_str = f"{int(day):02d}"
    except ValueError:
        raise ValueError("Month and Day must be convertible to integers.")

    # Construct the URL based on provided parameters
    url = f"https://api.sportradar.com/mlb/{access_level}/v7/{language_code}/games/{year}/{month_str}/{day_str}/summary.json"
    
    # Set headers to define expected response format
    headers = {'Accept': 'application/json'}
    
    # Send the API request
    response = requests.get(url, params={'api_key': api_key}, headers=headers)
    
    # Debug print to check URL and response status
    print(f"Request URL: {url} | Status Code: {response.status_code}")

    if response.status_code == 200:
        data = response.json()
        games = data.get('league', {}).get('games', [])
        print(f"Games found on {year}-{month_str}-{day_str}: {len(games)}")
        return [game['game']['id'] for game in games]
    elif response.status_code >= 500:
        print("Server error, consider retrying the request")
        return []
    else:
        print(f"Failed to retrieve data: Status Code {response.status_code}")
        return []

In [4]:
def get_game_summary(api_key, game_id):
    '''
    Fetches and extracts game summary information from the Sportradar API.

    Params:
    api_key: str - API key for accessing the Sportradar service
    game_id: str - Unique identifier for the game

    Returns:
    game: dict - Dictionary containing formatted game summary if successful, None otherwise
    '''
    url = f'https://api.sportradar.com/mlb/trial/v7/en/games/{game_id}/summary.json'
    response = requests.get(url, params={'api_key': api_key})

    # Initialize an empty dictionary to store game data
    game = {}

    # Check if the request was successful
    if response.status_code == 200:
        game_summary = response.json()
        
        # Accessing the 'game' key safely
        game_data = game_summary.get('game', {})
        
        # Extracting game details
        game['game_id'] = game_id
        game['away_team_abbr'] = game_data.get('away', {}).get('abbr', 'No abbreviation found')
        game['home_team_abbr'] = game_data.get('home', {}).get('abbr', 'No abbreviation found')
        
        # Handling scheduled time information
        scheduled_time = game_data.get('scheduled', 'No scheduled time found')
        game['date'] = scheduled_time.split('T')[0]
        game['start_time'] = scheduled_time.split('T')[1][:5] if 'T' in scheduled_time else 'Unknown'
        game['day_night'] = game_data.get('day_night', 'Unknown')

        # Extracting venue information
        venue_data = game_data.get('venue', {})
        game['venue'] = venue_data.get('name', 'No venue name found')
        game['field_type'] = venue_data.get('stadium_type', 'No stadium type found')

        # Statistics
        game['away_team_errors'] = game_data.get('away', {}).get('errors', 0)
        game['home_team_errors'] = game_data.get('home', {}).get('errors', 0)
        game['away_team_hits'] = game_data.get('away', {}).get('hits', 0)
        game['home_team_hits'] = game_data.get('home', {}).get('hits', 0)
        game['away_team_runs'] = game_data.get('away', {}).get('runs', 0)
        game['home_team_runs'] = game_data.get('home', {}).get('runs', 0)
        
        return game
    else:
        print(f"Failed to retrieve data for game ID {game_id}: Status Code {response.status_code}")
        return None

In [5]:
# Define the function to save game summaries to a CSV file
def save_game_summaries_to_csv(game_summaries, filename):
    # Define the CSV fieldnames based on the keys in the game summary dictionary
    fieldnames = game_summaries[0].keys() if game_summaries else []
    
    # Write the game summaries to the CSV file
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(game_summaries)

# Example usage
api_key = 'e5yITIoToh2mMp58F6Aym8kHrSe7pXbduUAME2nh'
access_level = 'trial'
language_code = 'en'
start_date = datetime(2024, 3, 1).date()

all_game_summaries = []
for single_date in daterange(start_date):
    year = single_date.year
    month = f"{single_date.month:02d}"  # Format as two digits
    day = f"{single_date.day:02d}"     # Format as two digits
    game_ids = get_daily_game_ids(api_key, access_level, language_code, year, month, day)
    for game_id in game_ids:
        game_summary = get_game_summary(api_key, game_id)
        if game_summary:
            all_game_summaries.append(game_summary)

# Save the game summaries to a CSV file
if all_game_summaries:
    filename = '../data/game_summaries.csv'
    for game_summary in all_game_summaries:
        df = pd.DataFrame([game_summary])
        df.to_csv(filename, mode='a', index=False, header=not os.path.exists(filename))
    print(f"Game summaries saved to {filename}")
else:
    print("No game summaries were collected.")

Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/03/01/summary.json | Status Code: 200
Games found on 2024-03-01: 16
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/03/02/summary.json | Status Code: 200
Games found on 2024-03-02: 16
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/03/03/summary.json | Status Code: 200
Games found on 2024-03-03: 16
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/03/04/summary.json | Status Code: 200
Games found on 2024-03-04: 12
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/03/05/summary.json | Status Code: 200
Games found on 2024-03-05: 13
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/03/06/summary.json | Status Code: 200
Games found on 2024-03-06: 13
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/03/07/summary.json | Status Code: 200
Games found on 2024-03-07: 13
Request URL: https://api.sportradar.com/mlb/trial/v7/en

Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/16/summary.json | Status Code: 200
Games found on 2024-04-16: 15
Failed to retrieve data for game ID aaea52c6-0022-43f3-98aa-6c59e991b2a1: Status Code 429
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/17/summary.json | Status Code: 200
Games found on 2024-04-17: 16
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/18/summary.json | Status Code: 200
Games found on 2024-04-18: 5
Failed to retrieve data for game ID 5bfed9c3-5208-4979-adc9-23d9ae5bc45f: Status Code 429
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/19/summary.json | Status Code: 200
Games found on 2024-04-19: 15
Failed to retrieve data for game ID c66741cc-0003-4a9e-bf81-0ed10df765c7: Status Code 429
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/20/summary.json | Status Code: 200
Games found on 2024-04-20: 16
Request URL: https://api.sportradar.com/mlb/trial/v7/en/